# [DONE] Changes to signal precision

Keeping precision part of signal is incorrect. This is because each pixel in the signal is the probability of corresponding class being recognized in a previous layer. That means the pixel value IS the confidence or precision. So, we should restructure the class hierarchy for signals and patterns to something like this -

*   A single Signal is a tensor that represents pattern machine confidence of previous layer. Maybe at sensory input layer, we need to do something special like distributed (quantized) representation.
*   SignalGrid is a grid of signals (tensors)
*   SignalSet is a set of SignalGrids
*   Pattern is a SignalSet for pixels and same shaped SignalSet for precision
*   PatternGrid is a grid of Patterns i.e. Grid(Union(SignalSet, SignalSet)] = Grid[Union({Grid(Tensor), Grid(Tensor), ...}, {Grid(Tensor), Grid(Tensor), ...}))

In [1]:
!cd /Users/amolk/work/AGI/pattern-machine && pip install --upgrade pip && pip install -r requirements.in && pip install -r dev-requirements.in

import sys
sys.path.append('/Users/amolk/work/AGI/pattern-machine/src')

  Using cached tox-3.23.1-py2.py3-none-any.whl (85 kB)
  Using cached pytest-6.2.4-py3-none-any.whl (280 kB)
  Using cached pytest_cov-2.12.0-py2.py3-none-any.whl (20 kB)
  Using cached mypy-0.812-cp38-cp38-macosx_10_9_x86_64.whl (9.1 MB)
  Using cached flake8-3.9.2-py2.py3-none-any.whl (73 kB)
  Using cached pep8_naming-0.11.1-py2.py3-none-any.whl (8.4 kB)
  Using cached black-21.5b2-py3-none-any.whl (138 kB)
  Using cached isort-5.8.0-py3-none-any.whl (103 kB)
  Using cached Sphinx-4.0.2-py3-none-any.whl (2.9 MB)
ERROR: Could not find a version that satisfies the requirement pyplot (from versions: none)
ERROR: No matching distribution found for pyplot


In [2]:
%load_ext autoreload
%autoreload 1
# %aimport pattern_machine.single_layer_regression

import numpy as np
import torch
from sklearn.datasets import load_boston
from tqdm.notebook import tqdm
from patternmachine.single_layer_regression import SingleLayerRegression

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def normalize(df):
  df1 = (df - df.mean())/df.std()
  return df1

def scale(df):
  min = df.min()
  max = df.max()

  df1 = (df - min) / (max - min)
  return df1

# dataset = load_boston()
# dataset = pd.DataFrame(dataset.data, columns=dataset.feature_names)
# dataset = pd.DataFrame(np.c_[scale(normalize(dataset['LSTAT'])), scale(normalize(dataset['RM']))], columns = ['LSTAT','RM'])
# dataset = torch.tensor(dataset.to_numpy()).float().to(device)
# dataset
# X = dataset[:,0]
# Y = dataset[:,1]

X = torch.linspace(0, 1, 100).unsqueeze(dim=-1)
Y = torch.linspace(0, 1, 100).unsqueeze(dim=-1)

pattern_machine = SingleLayerRegression(X, Y)
pattern_machine.layer.debug = False

# preset patterns with x and y as (0.0, 0.0) to (1.0, 1.0), i.e. y=x
for csgb in [pattern_machine.layer.patterns.pixels_begin, pattern_machine.layer.patterns.pixels_end]:
  csgb.components["x"].pixels = torch.linspace(0, 1, 10).unsqueeze(1)
  csgb.components["y"].pixels = torch.linspace(0, 1, 10).unsqueeze(1)

# present pattern precision 1.0
for csgb in [pattern_machine.layer.patterns.precision_begin, pattern_machine.layer.patterns.precision_end]:
  csgb.components["x"].pixels = torch.ones_like(csgb.components["x"].pixels)
  csgb.components["y"].pixels = torch.ones_like(csgb.components["y"].pixels)

print("pattern_machine.layer.patterns.composite_signal_grid_begin", pattern_machine.layer.patterns.pixels_begin.pixels)
pattern_machine.epoch()

 17%|█▋        | 17/100 [00:00<00:00, 164.19it/s]pattern_machine.layer.patterns.composite_signal_grid_begin {'x': tensor([[0.0000],
        [0.1111],
        [0.2222],
        [0.3333],
        [0.4444],
        [0.5556],
        [0.6667],
        [0.7778],
        [0.8889],
        [1.0000]]), 'y': tensor([[0.0000],
        [0.1111],
        [0.2222],
        [0.3333],
        [0.4444],
        [0.5556],
        [0.6667],
        [0.7778],
        [0.8889],
        [1.0000]]), '__output__': tensor([[0.0620, 0.0652, 0.0847, 0.0182, 0.0766, 0.0845, 0.0999, 0.0518, 0.0261,
         0.0480],
        [0.0184, 0.0074, 0.0591, 0.0050, 0.0962, 0.0543, 0.0132, 0.0625, 0.0510,
         0.0192],
        [0.0756, 0.0707, 0.0626, 0.0132, 0.0903, 0.0198, 0.0336, 0.0239, 0.0772,
         0.0924],
        [0.0956, 0.0733, 0.0860, 0.0073, 0.0927, 0.0537, 0.0748, 0.0672, 0.0966,
         0.0780],
        [0.0320, 0.0730, 0.0346, 0.0004, 0.0828, 0.0284, 0.0796, 0.0903, 0.0350,
         0.0383],
        

(0.08536447943221427, tensor(0.7768))